In [1]:
import os
import pandas as pd
import json

def get_split(replica_file):
    replica = json.load(open(replica_file, "r"))
    return replica["py/state"]["prepSettings"]["split"]['py/object'].split(".")[-1]

def get_results(data_folder, prefix):
    results = []
    for res_dir in [os.path.join(data_folder, x) for x in os.listdir(data_folder) if x.startswith(prefix)]:
        for res_file in [os.path.join(res_dir, x) for x in os.listdir(res_dir)]:
            if os.path.basename(res_file).startswith("results_"):
                res_df = pd.read_table(res_file)
                res_df["Split"] = res_df.ReplicaFile.apply(get_split)
                results.append(res_df)
    return pd.concat(results)

/tmp/ipykernel_2675424/2837410967.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# regression

In [3]:
df_reg_results = get_results("./data/", "regression")
df_reg_results.head()

,Assessor,ScoreFunc,Score,TargetProperty,TargetTask,ModelFile,Algorithm,AlgorithmParams,ReplicaID,DataSet,ReplicaFile,Split
0,TestSetAssessor,r2_score,-1.682650,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,STFullyConnected,NaN,regression_dnn_107420369,delaney-processed_MorganFP_gpu,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit
1,TestSetAssessor,root_mean_squared_error,3.308069,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,STFullyConnected,NaN,regression_dnn_107420369,delaney-processed_MorganFP_gpu,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit
2,TestSetAssessor,r2_score,-1.538419,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,STFullyConnected,NaN,regression_dnn_3184935163,delaney-processed_MorganFP_gpu,/home/martin/projects/qsp-bench-dev/data/regre...,ClusterSplit
3,TestSetAssessor,root_mean_squared_error,3.113120,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,STFullyConnected,NaN,regression_dnn_3184935163,delaney-processed_MorganFP_gpu,/home/martin/projects/qsp-bench-dev/data/regre...,ClusterSplit
4,TestSetAssessor,r2_score,-1.312194,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,STFullyConnected,NaN,regression_dnn_958682846,delaney-processed_MorganFP_gpu,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit


In [4]:
# classification (uncomment if ran as well)

In [5]:
# df_cls_results = get_results("./data/", "classification")
# df_cls_results.head()

In [6]:
df_reg_results["ScoreFuncNice"]  = df_reg_results.ScoreFunc.apply(lambda x: {"r2_score": r'$R^2$', "root_mean_squared_error": "RMSE"}[x])
df_reg_results = [
    df_reg_results[df_reg_results.Algorithm == "MedianDistributionAlgorithm"],
    df_reg_results[df_reg_results.Algorithm == "ChempropMoleculeModel"],
    df_reg_results[df_reg_results.Algorithm == "XGBRegressor"],
]
df_reg_results = pd.concat(df_reg_results)
df_reg_results

,Assessor,ScoreFunc,Score,TargetProperty,TargetTask,ModelFile,Algorithm,AlgorithmParams,ReplicaID,DataSet,ReplicaFile,Split,ScoreFuncNice
2,TestSetAssessor,r2_score,-0.000533,y,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,MedianDistributionAlgorithm,NaN,regression_478163327,freesolv_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit,$R^2$
3,TestSetAssessor,root_mean_squared_error,0.893743,y,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,MedianDistributionAlgorithm,NaN,regression_478163327,freesolv_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit,RMSE
6,TestSetAssessor,r2_score,-0.006679,y,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,MedianDistributionAlgorithm,NaN,regression_3184935163,freesolv_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,ClusterSplit,$R^2$
7,TestSetAssessor,root_mean_squared_error,0.863126,y,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,MedianDistributionAlgorithm,NaN,regression_3184935163,freesolv_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,ClusterSplit,RMSE
10,TestSetAssessor,r2_score,-0.002740,y,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,MedianDistributionAlgorithm,NaN,regression_1051802512,freesolv_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit,$R^2$
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,TestSetAssessor,root_mean_squared_error,1.751169,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,XGBRegressor,"{""n_jobs"": 1, ""random_state"": 3131575764}",regression_3131575764,delaney-processed_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,ClusterSplit,RMSE
232,TestSetAssessor,r2_score,0.731126,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,XGBRegressor,"{""n_jobs"": 1, ""random_state"": 701808367}",regression_701808367,delaney-processed_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit,$R^2$
233,TestSetAssessor,root_mean_squared_error,1.060222,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,XGBRegressor,"{""n_jobs"": 1, ""random_state"": 701808367}",regression_701808367,delaney-processed_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,RandomSplit,RMSE
236,TestSetAssessor,r2_score,0.418511,measured log solubility in mols per litre,REGRESSION,/home/martin/projects/qsp-bench-dev/data/regre...,XGBRegressor,"{""n_jobs"": 1, ""random_state"": 1629748727}",regression_1629748727,delaney-processed_MorganFP,/home/martin/projects/qsp-bench-dev/data/regre...,ClusterSplit,$R^2$


In [7]:
from utils.plotting import make_box_plot

def make_plot(df, prefix, out_dir):
    df["Data Set"] = df.DataSet.apply(lambda x: x.split("_")[0])
    # df = df[df.Algorithm != "STFullyConnected"].sort_values(by=["DataSetName", "Algorithm"])
    for split_name, df in df.groupby("Split"):
        make_box_plot(
            df,
            x="Data Set",
            y="Score",
            hue="Algorithm",
            plot_name=f"{prefix}{split_name}",
            ylim=(-0.5, 1),
            out_dir=out_dir,
            figsize=(8, 4.5),
            colors = [
                # "#DEF9C4",
                # "#9CDBA6",
                # "#50B498",
                # "#468585",
                "#FABC3F",
                "#E85C0D",
                "#C7253E",
            ]
        )

os.makedirs('./results_exp2', exist_ok=True)
make_plot(df_reg_results, "", out_dir='./results_exp2')
# make_plot(df_cls_results, "classification", out_dir='./results_exp2')

In [8]:
df_reg_results.to_csv('./results_exp2/regression.csv', header=True, index=False)
# df_cls_results.to_csv('./results_exp2/classification.csv', header=True, index=False)